In [ ]:
import pandas as pd
import numpy as np

In [ ]:
title_crew = pd.read_csv('datasets/title.crew.tsv', sep='\t', low_memory=False)
# drop writers column
title_crew = title_crew.drop(columns='writers')
# drop rows with multiple directors
title_crew = title_crew[~title_crew['directors'].str.contains(',')]
# drop rows with \N values (nan)
title_crew = title_crew[~title_crew['directors'].str.contains('N')]
# rename directors column to director
title_crew = title_crew.rename(columns={'directors' : 'director'})

In [ ]:
title_basics = pd.read_csv('datasets/title.basics.tsv', sep='\t', low_memory=False)
# join title_basics with title_crew on tconst
title_basics = title_basics.join(title_crew.set_index('tconst'), on='tconst')
# replace \N values with nan values
title_basics = title_basics.replace('\\N',np.NaN)
# drop rows with null value in the given rows
title_basics = title_basics.dropna(subset=['originalTitle', 'startYear', 'director', 'runtimeMinutes', 'genres'])
# drop endYear column, as it's null for most of the columns
title_basics = title_basics.drop(columns='endYear')
# get only the first genre
title_basics['genres'] = title_basics['genres'].apply(lambda x: x.split(',')[0])
# rename startYear and genres column
title_basics = title_basics.rename(columns={'startYear' : 'releaseYear', 'genres' : 'genre'})
# title_basics.to_csv('datasets/title_basics.tsv', sep='\t', index=False)

In [ ]:
title_ratings = pd.read_csv('datasets/title.ratings.tsv', sep='\t', low_memory=False)
# join title_basics with title_ratings on tconst
title_basics_ratings = title_basics.join(title_ratings.set_index('tconst'), on='tconst')
# drop rows with null values in rating columns
title_basics_ratings = title_basics_ratings.dropna(subset=['averageRating', 'numVotes'])

In [ ]:
name_basics = pd.read_csv('datasets/name.basics.tsv', sep='\t', low_memory=False)
# rename nconst column to director
name_basics = name_basics.rename(columns={'nconst' : 'director'})
# join titl_basics_ratings with name_basics on director
title_basics_ratings_director = title_basics_ratings.join(name_basics.set_index('director'), on='director')

In [ ]:
title_basics_ratings_director = pd.read_csv('datasets/title_basics_ratings_director.tsv', sep='\t', low_memory=False)
# drop rows with null values in primaryProfession column
title_basics_ratings_director = title_basics_ratings_director.dropna(subset=['primaryProfession'])
# replace multiple professions with one primary
title_basics_ratings_director['primaryProfession'] = title_basics_ratings_director['primaryProfession'].apply(lambda x: x.split(',')[0])
# replace multiple known for titles with one
title_basics_ratings_director['knownForTitles'] = title_basics_ratings_director['knownForTitles'].apply(lambda x: x.split(',')[0])
# rename columns
title_basics_ratings_director = title_basics_ratings_director.rename(columns={'tconst' : 'title_id', 'director' : 'director_id', 
    'primaryName' : 'director_primaryName', 'birthYear' : 'director_birthYear', 'deathYear' : 'director_deathYear', 
    'primaryProfession' : 'director_primaryProfession', 'knownForTitles': 'director_knownForTitle'})
title_basics_ratings_director['numVotes'] = title_basics_ratings_director['numVotes'].astype(np.int64)

In [ ]:
#separate data back to the original structures
title_basics = title_basics_ratings_director[['title_id', 'titleType', 'primaryTitle', 'originalTitle', 
    'isAdult', 'releaseYear', 'runtimeMinutes', 'genre', 'director_id']]
title_ratings = title_basics_ratings_director[['title_id', 'averageRating', 'numVotes']]
director_basics = title_basics_ratings_director[['director_id', 'director_primaryName', 'director_birthYear', 'director_deathYear', 
    'director_primaryProfession', 'director_knownForTitle']]

In [ ]:
# get 50000 rows from each dataframe
title_basics = title_basics.head(50000)
title_ratings = title_ratings.head(50000)
# drop duplicates in director_basics table
director_basics = director_basics.drop_duplicates(subset=['director_id'])
director_basics = director_basics.head(50000)

In [ ]:
# export the data
title_basics.to_csv('datasets/title_basics.csv', index=False)
title_ratings.to_csv('datasets/title_ratings.csv', index=False)
director_basics.to_csv('datasets/director_basics.csv', index=False)